In [3]:

%load_ext autoreload
%autoreload 2

In [26]:
#Get modules

from Headcuts import *
#localpath = pathlib.Path.cwd()

In [27]:
#HEADCUT#
#Import SEZ_Survey data from collect Do I need this?
#dfSEZ= get_sezsurvey_data()

#Get raw headcut data prior 2023 and earlier that is stored in gdbs in F:GIS/data/Monitoring/
#df= get_allheadcut_data()

#Get headcut data from sde collect
#df= get_newheadcut_data_collect()

df = get_combined_survey_and_headcut_data()

#clean, process, grade data
process_grade_headcut(df, 2024)

#post data to sde.VECTOR.stage headcut table or csv
post_headcut(df, draft=True)
